# **HYPERPARAMETER TUNING - GOOGLE COLAB CODE**

# Installing Keras Tuner

In [ ]:
!pip install -q -U keras-tuner

# Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tempfile
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt

# Load and Display Combined Data

In [ ]:
data_gabungan = pd.read_csv('/content/data_gabungan.csv')

In [ ]:
data_gabungan.head()

# Define Features (X) and Targets (Y)

In [ ]:
X = data_gabungan[['Close USDIDR', 'BI Rate', 'Inflasi']].values

Y = data_gabungan[['Close CTRA', 'Close INDF', 'Close ASII', 'Close BSDE',
                 'Close ICBP', 'Close KLBF', 'Close ITMG', 'Close JPFA',
                 'Close TLKM', 'Close ULTJ', 'Close ACES', 'Close TSPC',
                  'Close SMAR', 'Close SMSM', 'Close JRPT', 'Close DUTI',
                   'Close EPMT', 'Close SMCB', 'Close PWON', 'Close JSMR']].values

# Split Data into Training and Validation Sets and Standardize

In [ ]:
# 80% Train, 20% Val
total_ukuran_data = len(X)
ukuran_data_pelatihan = int(0.80 * total_ukuran_data)

X_Latih, Y_Latih = X[:ukuran_data_pelatihan], Y[:ukuran_data_pelatihan]
X_Validasi, Y_Validasi = X[ukuran_data_pelatihan:], Y[ukuran_data_pelatihan:]

skala_X = StandardScaler()
X_Latih_Standarisasi = skala_X.fit_transform(X_Latih)
X_Validasi_Standarisasi = skala_X.transform(X_Validasi)

skala_Y = StandardScaler()
Y_Latih_Standarisasi = skala_Y.fit_transform(Y_Latih)
Y_Validasi_Standarisasi = skala_Y.transform(Y_Validasi)

# Defining a Hyperparameter-Tuned Model Builder Function

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)  # Set up TPU strategy
    print("Running on TPU")
except ValueError:
    strategy = tf.distribute.get_strategy()  # Default strategy if TPU is not available
    print("Running on GPU/CPU")


In [ ]:
def model_builder(hp):
  with strategy.scope():
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Input(shape=(X_Latih.shape[1],)))
      num_layers = hp.Int('num_layers', min_value=2, max_value=6, step=1)
      for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=16, max_value=512, step=16)
        regularizer = hp.Float(f'kernel_regularizer_{i}', min_value=1e-6, max_value=1e-3, sampling='log')


        model.add(tf.keras.layers.Dense(units=units,
                                        activation='relu',
                                        kernel_regularizer=regularizers.l2(regularizer),
                                        ))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(rate=hp.Float(f'dropout_{i}', 0.2, 0.6, step=0.1)))

      model.add(tf.keras.layers.Dense(20, activation='linear'))
      
      optimizer_choice = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adamw'])

      if optimizer_choice == 'adam':
          hp_learning_rate = hp.Float('adam_learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')
          optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)

      elif optimizer_choice == 'sgd':
          hp_learning_rate = hp.Float('sgd_learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')
          hp_momentum = hp.Float('sgd_momentum', min_value=0.0, max_value=0.9, step=0.1)
          optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate, momentum=hp_momentum)

      elif optimizer_choice == 'rmsprop':
          hp_learning_rate = hp.Float('rmsprop_learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')
          hp_rho = hp.Float('rmsprop_rho', min_value=0.7, max_value=0.99, step=0.05)
          optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate, rho=hp_rho)

      elif optimizer_choice == 'adamw':
          hp_learning_rate = hp.Float('adamw_learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')
          hp_weight_decay = hp.Float('adamw_weight_decay', min_value=1e-6, max_value=1e-2, sampling='log')
          optimizer = tf.keras.optimizers.AdamW(learning_rate=hp_learning_rate, weight_decay=hp_weight_decay)

      model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

  return model

# Initializing the Hyperband Tuner for Model Optimization

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=100,
                     factor=2,
                     directory='TunnerV3',
                     project_name='stock_price_prediction',)

# Setting Up Early Stopping Callback

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=10,
                                              restore_best_weights=True
                                              )

# Running Hyperparameter Tuning with the Hyperband Tuner

In [ ]:
tuner.search(
    X_Latih_Standarisasi,
    Y_Latih_Standarisasi,
    validation_data=(X_Validasi_Standarisasi, Y_Validasi_Standarisasi),
    epochs=1000,
    callbacks=[stop_early],

)

# Retrieving and Displaying the Best Hyperparameters from Tuning Results

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print("Best Hyperparameters:")
for hp_name, hp_value in best_hps.values.items():
    print(f"{hp_name}: {hp_value}")

# Building and Displaying the Summary of the Optimized Model

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
model.summary()

# Setting Up Model Checkpointing to Save the Best Model

In [ ]:
checkpoint_filepath = 'best_model_v3.keras'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min')

# Training the Model with Checkpointing and Validation Data

In [ ]:
history = model.fit(X_Latih_Standarisasi, Y_Latih_Standarisasi,
                    validation_data=(X_Validasi_Standarisasi, Y_Validasi_Standarisasi),
                    epochs=1000,
                    batch_size=32,
                    callbacks=[checkpoint])

# Plotting Training and Validation Loss Over Epochs

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(loss) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs_range, loss, label='Training Loss', color='blue')
plt.plot(epochs_range, val_loss, label='Validation Loss', color='red')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.savefig("train.png", dpi=300)
plt.show()

# Loading the Best Model from Checkpoint

In [ ]:
best_model_v5 = tf.keras.models.load_model(checkpoint_filepath)

In [ ]:
best_model_v5.evaluate(X_Validasi_Standarisasi, Y_Validasi_Standarisasi)

In [ ]:
best_model_1 = tf.keras.models.load_model("/content/best_model_v1.keras")
best_model_2 = tf.keras.models.load_model("/content/best_model_v2.keras")
best_model_3 = tf.keras.models.load_model("/content/best_model_v3.keras")

print(best_model_1.evaluate(X_Validasi_Standarisasi, Y_Validasi_Standarisasi))
print(best_model_2.evaluate(X_Validasi_Standarisasi, Y_Validasi_Standarisasi))
print(best_model_3.evaluate(X_Validasi_Standarisasi, Y_Validasi_Standarisasi))

In [ ]:
Y_pred_1 = best_model_1(X_Validasi_Standarisasi).numpy()
Y_pred_2 = best_model_2(X_Validasi_Standarisasi).numpy()
Y_pred_3 = best_model_3(X_Validasi_Standarisasi).numpy()
Y_pred_1 = skala_Y.inverse_transform(Y_pred_1)
Y_pred_2 = skala_Y.inverse_transform(Y_pred_2)
Y_pred_3 = skala_Y.inverse_transform(Y_pred_3)
Y_Validasi = skala_Y.inverse_transform(Y_Validasi_Standarisasi)

num_stocks = Y_Validasi.shape[1]

In [ ]:
for i in range(num_stocks):
    plt.figure(figsize=(10, 6))
    plt.plot(Y_Validasi[:, i], label='Real Data')
    plt.plot(Y_pred_1[:, i], label='Prediction 1')
    plt.plot(Y_pred_2[:, i], label='Prediction 2')
    plt.plot(Y_pred_3[:, i], label='Prediction 3')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.title(f'Stock {i+1} Price Prediction') 
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
fig_all, axes_all = plt.subplots(num_stocks, 4, figsize=(24, num_stocks * 6), sharey=False)

for i in range(num_stocks):
    axes_all[i, 0].plot(Y_Validasi[:, i], label='Real Data', color='blue')
    axes_all[i, 0].set_title(f'Stock {i+1}: Actual')
    axes_all[i, 0].set_xlabel('Time')
    axes_all[i, 0].set_ylabel('Stock Price')
    axes_all[i, 0].grid(True)
    axes_all[i, 0].legend()

    axes_all[i, 1].plot(Y_pred_1[:, i], label='Prediction 1', color='green')
    axes_all[i, 1].set_title(f'Stock {i+1}: Prediction 1')
    axes_all[i, 1].set_xlabel('Time')
    axes_all[i, 1].grid(True)
    axes_all[i, 1].legend()

    axes_all[i, 2].plot(Y_pred_2[:, i], label='Prediction 2', color='red')
    axes_all[i, 2].set_title(f'Stock {i+1}: Prediction 2')
    axes_all[i, 2].set_xlabel('Time')
    axes_all[i, 2].grid(True)
    axes_all[i, 2].legend()

    axes_all[i, 3].plot(Y_pred_3[:, i], label='Prediction 3', color='orange')
    axes_all[i, 3].set_title(f'Stock {i+1}: Prediction 3')
    axes_all[i, 3].set_xlabel('Time')
    axes_all[i, 3].grid(True)
    axes_all[i, 3].legend()

plt.tight_layout()

output_path = "/content/stock_predictions_combined.png"
plt.savefig(output_path, dpi=300)
plt.show()

# Making Predictions on New Data and Displaying Stock Prices

In [ ]:
data_baru = np.array([[15569.15, 6.25, 2.12]])
data_baru_standarisasi = skala_X.transform(data_baru)

prediksi_standarisasi = model_terbaik.predict(data_baru_standarisasi)
prediksi = skala_Y.inverse_transform(prediksi_standarisasi)

saham = ['Close CTRA', 'Close INDF', 'Close ASII', 'Close BSDE', 'Close ICBP',
          'Close KLBF', 'Close ITMG', 'Close JPFA', 'Close TLKM', 'Close ULTJ','Close ACES', 'Close TSPC',
          'Close SMAR', 'Close SMSM', 'Close JRPT', 'Close DUTI', 'Close EPMT', 'Close SMCB', 'Close PWON', 'Close JSMR']
hasil_prediksi = dict(zip(saham, prediksi[0]))

print("Prediksi Harga Saham Berdasarkan Data Baru:")
for stock, price in hasil_prediksi.items():
    print(f"{stock}: {price:.2f}")


# Saving Standard Scalers to Disk Using Joblib

In [ ]:
import joblib
joblib.dump(skala_X, 'skala_X.pkl')
joblib.dump(skala_Y, 'skala_Y.pkl')

In [ ]:
X_mean = skala_X.mean_
X_std_dev = skala_X.scale_
Y_mean = skala_Y.mean_
Y_std_dev = skala_Y.scale_
print("Mean dari X (fitur):", X_mean)
print("Standard Deviation dari X (fitur):", X_std_dev)
print("Mean dari Y (target):", Y_mean)
print("Standard Deviation dari Y (target):", Y_std_dev)

# Exporting the Best Model to a Specified Directory

In [ ]:
model_path = "/content/best_model_v3.keras"
loaded_model = tf.keras.models.load_model(model_path)
export_path = "/content/best_model_v3_savedmodel"
tf.saved_model.save(loaded_model, export_path)

zip_filename = "/content/best_model_v3.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
  for root, _, files in os.walk(export_path):
    for file in files:
      zipf.write(os.path.join(root, file),
                 os.path.relpath(os.path.join(root, file),
                                 export_path))

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model /content/best_model_v3_savedmodel /content/tfjs_model

import zipfile
import os

zip_filename = "/content/tfjs_model.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
  for root, _, files in os.walk("/content/tfjs_model"):
    for file in files:
      zipf.write(os.path.join(root, file),
                 os.path.relpath(os.path.join(root, file),
                                 "/content/tfjs_model"))